In [1]:
# pip install ultralytics

In [2]:
# !pip install pandas

In [3]:
# !pip install scikit-learn

In [4]:
# !pip install xgboost

In [2]:
from ultralytics import YOLO
import os
import csv
import matplotlib.pyplot as plt
import cv2
import glob, cv2, os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import joblib

In [6]:
model = YOLO('best(Punch).pt')

In [7]:
folder = "P2_DATA_TRAIN"
output_folder = "predictions_with_bbox"  # โฟลเดอร์สำหรับเก็บภาพที่วาด bbox แล้ว
os.makedirs(output_folder, exist_ok=True)  # สร้างโฟลเดอร์ถ้ายังไม่มี

files = os.listdir(folder)
files = [x for x in files if x.lower().endswith(('.jpg', '.jpeg', '.png'))]

line_thickness = 1

with open("output.csv", "w", newline="", encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["image_file", "center_x", "center_y", "width", "height"])

for fname in files:
    image_path = os.path.join(folder, fname)
    
    original_image = cv2.imread(image_path)
    
    visualized_image = original_image.copy()
    
    results = model.predict(
        source=image_path,
        conf=0.01,
        iou=0.25,
        save=False,
        verbose=False
    )
    
    boxes = results[0].boxes
    
    if len(boxes) > 0:
        for box in boxes:
            xyxy = box.xyxy[0].cpu().numpy()
            x1, y1, x2, y2 = map(int, xyxy)  # แปลงเป็น integer สำหรับการวาด
            confidence = float(box.conf[0].cpu().numpy())

            center_x = (x1 + x2) / 2
            center_y = (y1 + y2) / 2
            width = x2 - x1
            height = y2 - y1
            
            
            # วาด bounding box ด้วย OpenCV โดยใช้ความหนาของเส้นที่กำหนด
            # สีเขียว (0, 255, 0) ในรูปแบบ BGR
            cv2.rectangle(visualized_image, (x1, y1), (x2, y2), 
                         (0, 255, 0), line_thickness)
            
            # วาดจุดกึ่งกลางเป็นวงกลมเล็กๆ สีแดง
            cv2.circle(visualized_image, (int(center_x), int(center_y)), 
                      3, (0, 0, 255), -1)
            
            label = f'Drone {confidence:.2f}'
            label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            
            # วาดพื้นหลังสำหรับข้อความให้อ่านง่าย
            cv2.rectangle(visualized_image,
                         (x1, y1 - label_size[1] - 10),
                         (x1 + label_size[0], y1),
                         (0, 255, 0), -1)
            
            # เขียนข้อความ
            cv2.putText(visualized_image, label, (x1, y1 - 5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
            
            row = [fname, round(center_x, 2), round(center_y, 2), 
                   round(width, 2), round(height, 2)]
            
            print("-" * 50)
            print(f"Image: {fname}")
            print(f"Center: ({center_x:.2f}, {center_y:.2f})")
            print(f"Size: {width:.2f} x {height:.2f}")
            
            with open("output.csv", "a", newline="", encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(row)
        
        # บันทึกภาพที่วาด bounding box แล้ว
        output_path = os.path.join(output_folder, fname)
        cv2.imwrite(output_path, visualized_image)
        print(f"Saved visualized image to: {output_path}")
    else:
        print(f"No drone detected in {fname}")

print("\nPrediction completed!")
print(f"Results saved to output.csv")

--------------------------------------------------
Image: img_0303.jpg
Center: (870.50, 428.50)
Size: 27.00 x 17.00
Saved visualized image to: predictions_with_bbox/img_0303.jpg
--------------------------------------------------
Image: img_0030.jpg
Center: (598.50, 408.50)
Size: 41.00 x 23.00
Saved visualized image to: predictions_with_bbox/img_0030.jpg
--------------------------------------------------
Image: img_0397.jpg
Center: (476.50, 618.00)
Size: 25.00 x 14.00
Saved visualized image to: predictions_with_bbox/img_0397.jpg
--------------------------------------------------
Image: img_0282.jpg
Center: (806.00, 349.50)
Size: 34.00 x 21.00
Saved visualized image to: predictions_with_bbox/img_0282.jpg
--------------------------------------------------
Image: img_0131.jpg
Center: (1206.00, 408.00)
Size: 16.00 x 10.00
Saved visualized image to: predictions_with_bbox/img_0131.jpg
--------------------------------------------------
Image: img_0152.jpg
Center: (872.00, 216.50)
Size: 16.00 x

In [8]:
# STEP 2: รวม Label จาก folder P2_DATA_TRAIN_
label_folder = "P2_DATA_TRAIN"
label_files = glob.glob(os.path.join(label_folder, "*.csv"))

label_list = []
for label_path in label_files:
    df = pd.read_csv(label_path)
    df['image_file'] = os.path.basename(label_path).replace('.csv', '.jpg')  # ปรับชื่อภาพให้ตรง
    label_list.append(df)


labels_df = pd.concat(label_list, ignore_index=True)
print(f"\nLoaded {len(labels_df)} label files")


Loaded 438 label files


In [9]:
# STEP 3: รวม YOLO features + Labels
features_df = pd.read_csv("output.csv")

# รวมโดยใช้ชื่อไฟล์ภาพเป็น key
merged_df = pd.merge(features_df, labels_df, on="image_file", how="inner")

print(f"Combined dataset size: {merged_df.shape}")
merged_df.to_csv("train_dataset.csv", index=False)
print("Saved merged dataset to train_dataset.csv")


Combined dataset size: (396, 8)
Saved merged dataset to train_dataset.csv


In [10]:
label_folder = "P2_DATA_TRAIN"
label_files = [f.replace(".csv", ".jpg") for f in os.listdir(label_folder) if f.endswith(".csv")]

# ภาพใน merged dataset ที่ตรงกับ label folder
df['label_exists'] = df['image_file'].isin(label_files)

# ตรวจสอบว่ามี image ไหน merge ผิด
mismatch = df[~df['label_exists']]
print("Images with no matching label:", mismatch['image_file'].tolist())

Images with no matching label: []


In [11]:
df = pd.read_csv('train_dataset.csv') 
print(df)

       image_file  center_x  center_y  width  height   Latitude   Longitude  \
0    img_0303.jpg     870.5     428.5     27      17  14.304723  101.172726   
1    img_0030.jpg     598.5     408.5     41      23  14.304737  101.172762   
2    img_0397.jpg     476.5     618.0     25      14  14.304630  101.172704   
3    img_0282.jpg     806.0     349.5     34      21  14.304775  101.172800   
4    img_0131.jpg    1206.0     408.0     16      10  14.304289  101.171699   
..            ...       ...       ...    ...     ...        ...         ...   
391  img_0422.jpg     520.5     484.5     43      25  14.304762  101.172788   
392  img_0275.jpg     759.5     410.0     49      28  14.304812  101.172856   
393  img_0358.jpg     544.0     277.0     22      14  14.304526  101.172626   
394  img_0418.jpg     401.5     440.0     31      18  14.304685  101.172769   
395  img_0418.jpg     793.5     375.0     27      16  14.304685  101.172769   

     Altitude  
0       59.12  
1       47.29  
2  

## FE

## เพิ่มฟีเจอร์ใหม่จาก YOLO output

In [12]:
IMAGE_WIDTH = 1920
IMAGE_HEIGHT = 1080
# สัดส่วนของวัตถุ ช่วยบอกระยะทางหรือมุมกล้อง
df['aspect_ratio'] = df['width'] / df['height']  
# พื้นที่ของวัตถุในภาพ มักสัมพันธ์กับ “ความใกล้/ไกล”
df['area'] = df['width'] * df['height']
# ทำให้ตำแหน่งอยู่ในช่วง [0,1]
df['norm_x'] = df['center_x'] / IMAGE_WIDTH    # เช่น 1280
# ทำให้ตำแหน่งอยู่ในช่วง [0,1]
df['norm_y'] = df['center_y'] / IMAGE_HEIGHT   # เช่น 720
# พื้นที่กล่องเชิงสัมพัทธ์
df['norm_area'] = df['area'] / (IMAGE_WIDTH * IMAGE_HEIGHT)
# ระยะห่างจากศูนย์กลางภาพในแนวนอน
df['delta_x'] = df['norm_x'] - 0.5
# ระยะห่างจากศูนย์กลางภาพในแนวตั้ง
df['delta_y'] = df['norm_y'] - 0.5

## แปลงพิกัดกล้องและเป้าหมายให้อยู่ในหน่วยเมตร (Local Coordinates)

In [13]:
import numpy as np

lat0, lon0 = 14.305029, 101.173010  # กล้อง
R = 6371000  # รัศมีโลก (m)
# ระยะทางแนวนอน (ตะวันออก–ตก) จากกล้อง เมตร
df['target_x'] = (df['Longitude'] - lon0) * np.cos(np.radians(lat0)) * 111320
# ระยะทางแนวเหนือ–ใต้จากกล้อง เมตร
df['target_y'] = (df['Latitude'] - lat0) * 110540
# ความสูงของโดรนเทียบกับกล้อง เมตร
df['target_z'] = df['Altitude'] - 37.2  # ความสูงสัมพัทธ์กับกล้อง

## รวมฟีเจอร์กล้อง (ค่าคงที่ในกรณีนี้)

In [14]:
# พิกัดจริงของกล้อง (ละติจูด, ลองจิจูด)
df['camera_lat'] = lat0
df['camera_lon'] = lon0
df['camera_alt'] = 37.2

In [15]:
# 1.1. มุมเบี่ยง (เชิง field of view) — สมมุติว่ากล้องมี FOV แนวนอน 70°, แนวตั้ง 50°
FOV_X = np.deg2rad(70)
FOV_Y = np.deg2rad(50)

# คำนวณมุมเบี่ยงของโดรนในภาพ (เป็นหน่วยเรเดียน)
df['theta_x'] = (df['norm_x'] - 0.5) * FOV_X
df['theta_y'] = (0.5 - df['norm_y']) * FOV_Y

# 1.2. ความสูงต่างระหว่างโดรนกับกล้อง
df['alt_diff'] = df['Altitude'] - df['camera_alt']

# 1.3. ระยะทางเชิงประมาณจากขนาด bounding box (กล่องเล็ก = ไกล)
df['inv_area'] = 1.0 / (df['norm_area'] + 1e-6)   # +1e-6 เพื่อกันหารศูนย์

# 1.4. อัตราส่วนของขนาดกล่องเทียบกับจุดศูนย์กลาง (เพื่อชี้ว่าโดรนอยู่ใกล้-ไกลแค่ไหนในแนวลึก)
df['size_depth_ratio'] = df['norm_area'] / (np.abs(df['delta_x']) + np.abs(df['delta_y']) + 1e-6)

# 1.5. ระยะ offset จากศูนย์ภาพ (รวมทั้งแนว x และ y)
df['offset_mag'] = np.sqrt(df['delta_x']**2 + df['delta_y']**2)

In [16]:
print(df)

       image_file  center_x  center_y  width  height   Latitude   Longitude  \
0    img_0303.jpg     870.5     428.5     27      17  14.304723  101.172726   
1    img_0030.jpg     598.5     408.5     41      23  14.304737  101.172762   
2    img_0397.jpg     476.5     618.0     25      14  14.304630  101.172704   
3    img_0282.jpg     806.0     349.5     34      21  14.304775  101.172800   
4    img_0131.jpg    1206.0     408.0     16      10  14.304289  101.171699   
..            ...       ...       ...    ...     ...        ...         ...   
391  img_0422.jpg     520.5     484.5     43      25  14.304762  101.172788   
392  img_0275.jpg     759.5     410.0     49      28  14.304812  101.172856   
393  img_0358.jpg     544.0     277.0     22      14  14.304526  101.172626   
394  img_0418.jpg     401.5     440.0     31      18  14.304685  101.172769   
395  img_0418.jpg     793.5     375.0     27      16  14.304685  101.172769   

     Altitude  aspect_ratio  area  ...  target_z  c

In [17]:
print(df.columns.tolist())

['image_file', 'center_x', 'center_y', 'width', 'height', 'Latitude', 'Longitude', 'Altitude', 'aspect_ratio', 'area', 'norm_x', 'norm_y', 'norm_area', 'delta_x', 'delta_y', 'target_x', 'target_y', 'target_z', 'camera_lat', 'camera_lon', 'camera_alt', 'theta_x', 'theta_y', 'alt_diff', 'inv_area', 'size_depth_ratio', 'offset_mag']


In [21]:
# ฟีเจอร์ที่ใช้
X_cols = [
    'norm_x', 'norm_y', 'norm_area', 'aspect_ratio',
    'delta_x', 'delta_y',
    'theta_x', 'theta_y',
    'inv_area', 'alt_diff',
    'size_depth_ratio', 'offset_mag'
]

# label
y_cols = ['target_x', 'target_y', 'target_z']

# ดึงข้อมูลจริงจาก df
X = df[X_cols]
y = df[y_cols]

# แบ่ง train/test
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2)

In [35]:
# สมมติค่าพิกัดกล้องเริ่มต้น
lat0 = 14.305029
lon0 = 101.173010
alt0 = 37.2

models = {}

# เทรนโมเดลและประเมินแกน x,y,z ในหน่วยเมตร
for axis in ['target_x', 'target_y', 'target_z']:
    model = RandomForestRegressor(n_estimators=300)
    model.fit(X_train, y_train[axis])
    models[axis] = model
    
    pred = model.predict(X_test)
    mae = mean_absolute_error(y_test[axis], pred)
    rmse = np.sqrt(mean_squared_error(y_test[axis], pred))
    r2 = r2_score(y_test[axis], pred)
    print(f"{axis} - MAE: {mae:.2f} m, RMSE: {rmse:.2f} m, R²: {r2:.3f}")

# บันทึก dictionary ของโมเดล
joblib.dump(models, 'models_all.pkl')

# โหลดกลับมาใช้
models_loaded = joblib.load('models_all.pkl')
# pred_x = models_loaded['target_x'].predict(X_test)

# ทำนายทั้งหมดพร้อมกัน (offset ในหน่วยเมตร)
pred_x = models['target_x'].predict(X_test)
pred_y = models['target_y'].predict(X_test)
pred_z = models['target_z'].predict(X_test)

# แปลงค่าพิกัด local offset เป็น Latitude, Longitude, Altitude จริง
pred_lat = lat0 + (pred_y / 110540)
pred_lon = lon0 + (pred_x / (111320 * np.cos(np.radians(lat0))))
pred_alt = alt0 + pred_z

# รวมพิกัดจริงจาก y_test แปลงกลับเป็น lat, lon, alt
true_lat = lat0 + (y_test['target_y'] / 110540)
true_lon = lon0 + (y_test['target_x'] / (111320 * np.cos(np.radians(lat0))))
true_alt = alt0 + y_test['target_z']

# คำนวณ error ในพิกัด lat, lon, alt
mae_lat = mean_absolute_error(true_lat, pred_lat)
rmse_lat = np.sqrt(mean_squared_error(true_lat, pred_lat))
r2_lat = r2_score(true_lat, pred_lat)

mae_lon = mean_absolute_error(true_lon, pred_lon)
rmse_lon = np.sqrt(mean_squared_error(true_lon, pred_lon))
r2_lon = r2_score(true_lon, pred_lon)

mae_alt = mean_absolute_error(true_alt, pred_alt)
rmse_alt = np.sqrt(mean_squared_error(true_alt, pred_alt))
r2_alt = r2_score(true_alt, pred_alt)

print("\n--- Evaluation on Latitude/Longitude/Altitude ---")
print(f"Latitude  - MAE: {mae_lat:.7f}°, RMSE: {rmse_lat:.7f}°, R²: {r2_lat:.3f}")
print(f"Longitude - MAE: {mae_lon:.7f}°, RMSE: {rmse_lon:.7f}°, R²: {r2_lon:.3f}")
print(f"Altitude  - MAE: {mae_alt:.2f} m, RMSE: {rmse_alt:.2f} m, R²: {r2_alt:.3f}")

# แสดงตัวอย่างพิกัดทำนาย
df_pred = pd.DataFrame({
    'pred_latitude': pred_lat,
    'pred_longitude': pred_lon,
    'pred_altitude': pred_alt
}, index=y_test.index)

print("\nตัวอย่างพิกัดที่แปลงจาก prediction:")
# ตั้งค่าให้ pandas แสดงทุกแถว ทุกคอลัมน์ โดยไม่ตัด ...
pd.set_option('display.max_rows', None)     # แสดงทุกรายการแถว
pd.set_option('display.max_columns', None)  # แสดงทุกคอลัมน์
pd.set_option('display.width', None)        # ขยายความกว้างไม่ตัดบรรทัด
pd.set_option('display.max_colwidth', None) # แสดงความกว้างเต็มของคอลัมน์

lat0 = 14.305029
lon0 = 101.173010
alt0 = 37.2

# แปลง target offset เป็นพิกัดจริง
true_lat = lat0 + (y_test['target_y'] / 110540)
true_lon = lon0 + (y_test['target_x'] / (111320 * np.cos(np.radians(lat0))))
true_alt = alt0 + y_test['target_z']

# สร้าง DataFrame สำหรับค่าจริงที่แปลงแล้ว
df_true = pd.DataFrame({
    'true_latitude': true_lat,
    'true_longitude': true_lon,
    'true_altitude': true_alt
}, index=y_test.index)

# รวมกับค่าที่ทำนายไว้แล้วใน df_pred
df_compare = pd.concat([df_pred, df_true], axis=1)

print(df_compare)


target_x - MAE: 4.73 m, RMSE: 7.64 m, R²: 0.950
target_y - MAE: 3.46 m, RMSE: 5.86 m, R²: 0.921
target_z - MAE: 0.17 m, RMSE: 0.42 m, R²: 0.999

--- Evaluation on Latitude/Longitude/Altitude ---
Latitude  - MAE: 0.0000313°, RMSE: 0.0000530°, R²: 0.921
Longitude - MAE: 0.0000439°, RMSE: 0.0000708°, R²: 0.950
Altitude  - MAE: 0.17 m, RMSE: 0.42 m, R²: 0.999

ตัวอย่างพิกัดที่แปลงจาก prediction:
     pred_latitude  pred_longitude  pred_altitude  true_latitude  \
144      14.304749      101.172709      52.229167      14.304718   
17       14.304766      101.172797      57.195533      14.304783   
60       14.304639      101.172809      54.389767      14.304638   
87       14.304904      101.172513      58.984867      14.304887   
21       14.304854      101.172596      53.811133      14.304834   
76       14.304689      101.172750      48.313800      14.304723   
355      14.304818      101.172254      49.984767      14.304848   
179      14.304781      101.172833      45.114467      14.304

In [1]:
#pred_x = models['target_x'].predict(X_new)
#pred_y = models['target_y'].predict(X_new)
#pred_z = models['target_z'].predict(X_new)

#pred_lat = lat0 + (pred_y / 110540)
#pred_lon = lon0 + (pred_x / (111320 * np.cos(np.radians(lat0))))
#pred_alt = 37.2 + pred_z